# Proof of concept search module

The idea with this module is to Google search 100+ results, then use an LLM to organize and re-rank the results.

## Ideas to consider

- Multiple searches: The company, company + product, search for product reviews, search all time vs just recent
- Provide a markdown template to fill out with an "other" category, and refine the "other" listings

In [12]:
from core import CompanyProduct, init_requests_cache, init_langchain_cache, make_experiment_dir

init_requests_cache()
init_langchain_cache()

'/home/keith/company-detective/.cache/langchain.sqlite'

In [29]:
target = CompanyProduct("Singularity 6", "Palia")
experiment_dir = make_experiment_dir(target)

In [34]:
from search import search
from pprint import pprint

search_results = list(search(f'"{target.company}"', num=100))
if target.product != target.company:
    search_results += list(search(f'"{target.company}" "{target.product}"', num=100))
pprint(search_results)

[SearchResult(title='Singularity 6', link='https://www.singularity6.com/', snippet="Based out of Los Angeles, Singularity 6 is a video game developer founded with the mission to forge alternate worlds that deepen players' lives.", formattedUrl='https://www.singularity6.com/'),
 SearchResult(title='Singularity 6 is not an indie company : r/Palia', link='https://www.reddit.com/r/Palia/comments/15ss9gg/singularity_6_is_not_an_indie_company/', snippet='Aug 16, 2023 ... 180 votes, 81 comments. Singularity 6 is a multi million dollar studio with over 50 million dollar investments made.', formattedUrl='https://www.reddit.com/r/Palia/.../singularity_6_is_not_an_indie_company/'),
 SearchResult(title='Daybreak acquires struggling Palia developer Singularity 6', link='https://www.gamedeveloper.com/business/daybreak-acquires-struggling-palia-developer-singularity-6', snippet='Jul 2, 2024 ... Singularity 6 has conducted multiple rounds of layoffs since the turn of the year, but will now join the Da

In [35]:
from typing import List
from search import SearchResult

def result_to_markdown(search_result: SearchResult) -> str:
    return f"[{search_result.title}]({search_result.link})\n{search_result.snippet}"

def results_to_markdown(search_results: List[SearchResult]) -> str:
    return "\n\n".join(result_to_markdown(result) for result in search_results)

print(results_to_markdown(search_results))

[Singularity 6](https://www.singularity6.com/)
Based out of Los Angeles, Singularity 6 is a video game developer founded with the mission to forge alternate worlds that deepen players' lives.

[Singularity 6 is not an indie company : r/Palia](https://www.reddit.com/r/Palia/comments/15ss9gg/singularity_6_is_not_an_indie_company/)
Aug 16, 2023 ... 180 votes, 81 comments. Singularity 6 is a multi million dollar studio with over 50 million dollar investments made.

[Daybreak acquires struggling Palia developer Singularity 6](https://www.gamedeveloper.com/business/daybreak-acquires-struggling-palia-developer-singularity-6)
Jul 2, 2024 ... Singularity 6 has conducted multiple rounds of layoffs since the turn of the year, but will now join the Daybreak family.

[Palia developers, Singularity 6, axes 35% of staff just after Steam ...](https://www.reddit.com/r/MMORPG/comments/1bz2e0z/palia_developers_singularity_6_axes_35_of_staff/)
Apr 9, 2024 ... Palia developers, Singularity 6, axes 35% of s

In [36]:
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages.ai import AIMessage
from langchain_openai import ChatOpenAI

from core import CompanyProduct
from dotenv import load_dotenv

load_dotenv()


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You're an expert at organizing search results.
Given search results for a company or product, organize them into the following headers:

# Official social media
# Job boards
# App stores
# Product reviews
# News articles (most recent first, grouped by event)
# Key employees (with subheaders by employee)
# Other pages on the company website
# Other

Include the publication date after the link, if available.

Unless otherwise specified, order the results in each section from most to least relevant.
Format the output as a markdown document, preserving any links in the source.
Organize ALL search results into these headers; do not omit any results.
            """,
        ),
        (
            "human",
            """
            Company: {company_name}
            Product: {product_name}
            
            Search results: 
            {text}
            """,
        ),
    ]
)


def summarize(
    target: CompanyProduct, search_results: List[SearchResult], debug=True
) -> AIMessage:
    unified_markdown = results_to_markdown(search_results)

    if debug:
        print(f"{len(unified_markdown):,} characters in unified context")

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    runnable = prompt | llm
    result = runnable.invoke({"text": unified_markdown, "company_name": target.company, "product_name": target.product})
    result.content = result.content.strip().strip("```markdown").strip("```")
    return result

summary = summarize(target, search_results)
print(summary.content)

with open(f"{experiment_dir}/search_results.md", "w") as f:
    f.write(summary.content)

    f.write("\n# Sources\n")
    for result in search_results:
        f.write(result_to_markdown(result) + "\n\n")

47,387 characters in unified context
# Official social media
- [Singularity 6 | LinkedIn](https://www.linkedin.com/company/singularity6)  
- [Singularity 6 (@Singularity6) / X](https://twitter.com/singularity6?lang=en)  
- [Singularity 6 | Facebook](https://www.facebook.com/singularity6/)  

# Job boards
- [Singularity 6 Los Angeles Office: Careers, Perks + Culture | Built In](https://www.builtinla.com/company/singularity-6)  
- [Singularity 6 People | Wellfound (formerly AngelList Talent)](https://wellfound.com/company/singularity6/people)  
- [Singularity 6 Employee Directory, Headcount & Staff | LeadIQ](https://leadiq.com/c/singularity-/5c5264831f0000001d596fec/employee-directory)  

# App stores
- [Palia on Steam](https://store.steampowered.com/app/2707930/Palia/)  
- [Palia for Nintendo Switch - Nintendo Official Site](https://www.nintendo.com/us/store/products/palia-switch/)  
- [Palia | Download and Play for Free - Epic Games Store](https://store.epicgames.com/en-US/p/palia-0d42